In [4]:
import numpy as np
!pip install scikit-fuzzy
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider


# --- 1. Universe & Membership Functions ---
suhu = np.arange(0, 41, 1)
kelembapan = np.arange(0, 101, 1)
kipas = np.arange(0, 101, 1)

# Suhu
suhu_dingin = fuzz.trimf(suhu, [0, 0, 20])
suhu_hangat = fuzz.trimf(suhu, [15, 25, 35])
suhu_panas = fuzz.trimf(suhu, [30, 40, 40])

# Kelembapan
hum_rendah = fuzz.trimf(kelembapan, [0, 0, 40])
hum_sedang = fuzz.trimf(kelembapan, [30, 50, 70])
hum_tinggi = fuzz.trimf(kelembapan, [60, 100, 100])

# Kipas
kipas_pelan = fuzz.trimf(kipas, [0, 0, 50])
kipas_sedang = fuzz.trimf(kipas, [25, 50, 75])
kipas_cepat = fuzz.trimf(kipas, [50, 100, 100])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 11.5 MB/s eta 0:00:00


In [5]:
# --- 2. Fungsi Interaktif ---
def fuzzy_mamdani(suhu_val, kelembapan_val):
    # Fuzzifikasi
    mu_suhu_dingin = fuzz.interp_membership(suhu, suhu_dingin, suhu_val)
    mu_suhu_hangat = fuzz.interp_membership(suhu, suhu_hangat, suhu_val)
    mu_suhu_panas = fuzz.interp_membership(suhu, suhu_panas, suhu_val)

    mu_hum_rendah = fuzz.interp_membership(kelembapan, hum_rendah, kelembapan_val)
    mu_hum_sedang = fuzz.interp_membership(kelembapan, hum_sedang, kelembapan_val)
    mu_hum_tinggi = fuzz.interp_membership(kelembapan, hum_tinggi, kelembapan_val)

    # --- 3. Aturan Fuzzy (Mamdani) ---
    # Rule 1: If Suhu dingin OR Kelembapan rendah → Kipas pelan
    alpha1 = np.fmax(mu_suhu_dingin, mu_hum_rendah)
    # Rule 2: If Suhu hangat → Kipas sedang
    alpha2 = mu_suhu_hangat
    # Rule 3: If Suhu panas OR Kelembapan tinggi → Kipas cepat
    alpha3 = np.fmax(mu_suhu_panas, mu_hum_tinggi)

    # --- 4. Implikasi (clipping) ---
    pelan_clipped = np.fmin(alpha1, kipas_pelan)
    sedang_clipped = np.fmin(alpha2, kipas_sedang)
    cepat_clipped = np.fmin(alpha3, kipas_cepat)

    # --- 5. Agregasi Output ---
    aggregated = np.fmax(pelan_clipped, np.fmax(sedang_clipped, cepat_clipped))

    # --- 6. Defuzzifikasi dengan beberapa metode ---
    centroid = fuzz.defuzz(kipas, aggregated, 'centroid')
    bisector = fuzz.defuzz(kipas, aggregated, 'bisector')
    mom = fuzz.defuzz(kipas, aggregated, 'mom')

    # --- 7. Visualisasi ---
    plt.figure(figsize=(8, 4))
    plt.plot(kipas, kipas_pelan, 'b--', label='Pelan')
    plt.plot(kipas, kipas_sedang, 'g--', label='Sedang')
    plt.plot(kipas, kipas_cepat, 'r--', label='Cepat')
    plt.fill_between(kipas, aggregated, alpha=0.4, color='orange', label='Aggregated')

    plt.axvline(centroid, color='k', linestyle='--', label=f'Centroid = {centroid:.1f}')
    plt.axvline(bisector, color='purple', linestyle='--', label=f'Bisector = {bisector:.1f}')
    plt.axvline(mom, color='brown', linestyle='--', label=f'MOM = {mom:.1f}')

    plt.title(f"Suhu={suhu_val:.1f}°C, Kelembapan={kelembapan_val:.1f}%")
    plt.xlabel("Kecepatan Kipas (%)")
    plt.ylabel("Derajat Keanggotaan")
    plt.ylim(-0.05, 1.05)
    plt.grid(True)
    plt.legend(loc='upper left', bbox_to_anchor=(1.01, 1))
    plt.show()

    # Print hasil defuzzifikasi
    print(f"Defuzzifikasi:")
    print(f"  Centroid  = {centroid:.2f}%")
    print(f"  Bisector  = {bisector:.2f}%")
    print(f"  MOM       = {mom:.2f}%")

In [6]:
# --- 8. Widget interaktif ---
interact(
    fuzzy_mamdani,
    suhu_val=FloatSlider(value=30, min=0, max=40, step=1, description='Suhu (°C)'),
    kelembapan_val=FloatSlider(value=70, min=0, max=100, step=1, description='Kelembapan (%)')
)

interactive(children=(FloatSlider(value=30.0, description='Suhu (°C)', max=40.0, step=1.0), FloatSlider(value=…

<function __main__.fuzzy_mamdani(suhu_val, kelembapan_val)>